In [ ]:
import pytorch as nn
import numpy as np
import lightning as L
import copy as copy

#sublayer

In [ ]:
class sublayerconnection(nn.Modile):
  def __init__(self.sice.dropoat):
    super(sublayerconnection,self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)


  def forward(self,x,sublayer):
    "Apply residual connection to any sublayer function that maintains the same size"
    return x + self.dropout(sublayer(self.norm(x)))



#clones

In [ ]:
def clones(module,N):
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

#layer normaliztion

In [ ]:
class LayerNorm(nn.module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


#Decoder

In [ ]:
class DecoderLayer(nn.module):
    def __init__(self, size, self_attn,src_attn,feed_forward,dropout):
      supor(DecoderLayer,self).__init__()
      self.size = size
      self.self_attn = self_attn
      self.src_attn = src_attn
      self.feed_forward = feed_forward
      self.sublayer = clones(sublayerconnection(size,dropout),3)

    def forward(self,x,memory,src_mask,tgt_mask):
      m = memory
      x = self.sublayer[0](x,lambda x:self.self_attn(x,x,x,tgt_mask))
      x = self.sublayer[1](x. lambda x: self.src_attn(x,m,m,src_mask))
      return self.sublayer[2](x,self.feed_forward)

class Decoder(nn.module):
  def __init__(sef,layer,N):
    super(Decoder,self).__init__()
    self.layers = clones(layer,N)
    self.norm = LayerNorm(layer.size)

  def forward(self,x,memory,src_mask,tgt_mask):
    for layer in self.layers:
      x = layer(x,memory,src_mask,tgt_mask)
    return self.norm(x)

#Encoder

In [ ]:
class EncoderLayer(nn.module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self.attn
        self.feed_forward = feed_forward
        self.sublayer = clones(sublayerconnection(size,droput),2)
        self.size = size

    def forward(self,x,mask):
      x = self.sublayer[0](x,lambda x:self.self_attn(x,x,x,mask))
      return self.sublayer[1](x,self.feed_forward)

class Encoder(nn.module):
  def __init__(self,layer,N):
    super(Encoder,self).__init__()
    self.layers = clones(layer,N)
    self.norm = LayerNorm(layer.size)

  def forward(self,x,mask):
    for layer in self.layers:
      x = layer(x,mask)
     return self.norm(x)


#Masking future values

In [ ]:
def subsequent_mask(size):
  attn_shape = (1,size,size)
  subsequent_mask = np.triu(np.ones(attn_shape),k=1).astype('unit8')
  return torch.from_numpy(subsequent_mask) == 0

#Encoder - Decoder

In [ ]:
class EncoderDecoder(L.LightningModule):
  def __init__(self,encoder,decoder,soarce_embed,target_embed,generator,criterion):
    super(EncoderDecoder,self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = soarce_embed
    self.tgt_embed = target_embed
    self.generator = generator
    self.criterion =criterion

  def encode(self,sre,src_mask):
    return self.encoder(self.src_embed(src),src_mask)

  def decode(self,memory,src_mask,tgt,tgt_mask):
    return self.decoder(self.tgt_embed(tgt),memory,src_mask,tgt_mask)

  def forward(self,src,tgt,src_mask,tgt_mask):
    return self.decode(self.encode(src,src_mask),src_mask,tgt,tgt_mask)

  # def generator_score(self,logits):
  #   return F.log_softmax(logits,dim=-1

#Attention

In [ ]:
def attention(query,key,value,mask=None,dropout=None):
  d_k = query.size(-1)
  scores = torch.matmul(query,key.transpose(-1,-2))/math.sqrt(d_k)
  if mask is not None:
    scores = scores.masked_fill(mask==0,-1e9)
  p_attn = F.softmax(scores,dim=-1)
  if dropout is not None:
    p_attn = dropout(p_attn)
  return torch.matmul(p_attn,value),p_attn

#Multi Head Attention

In [ ]:
class MultiHeadedAttention(nn.Module):
  def __init__(self,h,d_model,dropout=0.1):
    super(MultiHeadedAttention,self).__init__()
    assert d_model % h == 0
    self.d_k = d_model // h
    self.h = h
    self.linears = clones(nn.Linear(d_model,d_model),4)
    self.attn = None
    self.dropout = nn.Dropout(p=dropout)

  def forward(self,query,key,value,mask=None):
    if mask is not None:
      mask = mask.unsqueeze(1)
    nbatches = query.size(0)
    query,key,value = [l(x).view(nbatches,-1,self.h,self.d_k).transpose(1,2) for l,x in zip(self.linears,(query,key,value))]
    x , self.attn = attention(query,key,value,mask=mask,dropout=self.dropout)
    x = x.transpose(1,2).contiguous().view(nbatches,-1,self.h*self.d_k)
    return self.linears[-1](x)



#position- wise feed forward

In [ ]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self,d_model,d_ff,dropout=0.1):
    super(PositionwiseFeedForward,self).__init__()
    self.w_1 = nn.Linear(d_model,d_ff)
    self.w_2 = nn.Linear(d_ff,d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    return self.w_2(self.dropout(F.relu(self.w_1(x))))

#Embeddings

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)  # look up matrix
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

#positional encoding

In [ ]:
class position_encoding(nn.Module):
  def __init__(self,d_model,dropout,max_len=1000):
    super(position_encoding,self).__init__()
    self.dropout = nn.Dropout(p=dropout)
    pe = torch.zeros(max_len,d_model)
    position = torch.arange(0,max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0,d_model,2)*-(math.log(10000.0)/d_model))
    pe[::2]=torch.sin(position*div_term)
    pe[1::2]=torch.cos(position*div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer('pe',pe)

  def forward(self,x):
    x = x + Variable(self.pe[:,:x.size(1)],requires_grad=False)
    return self.dropout(x)


#Label Smoothing

In [ ]:
class LabelSmoothing(nn.Module):
  def __init__(self,size,padding_idx,smoothing=0.0):
    super(LabelSmoothing,self).__init__()
    self.criterion = nn.KLDivLoss(size_average=False)
    self.padding_idx = padding_idx
    self.confidence = 1.0 - smoothing
    self.smoothing = smoothing
    self.size = size
    self.true_dist = None

  def forward(self,x,target):
    assert x.size(1) == self.size
    true_dist = x.data.clone()
    true_dist.fill_(self.smoothing/(self.size-2))
    turt_dist.scatter_(1,target.data.unsqueeze(1),self.confidence)
    true_dist[:,self.padding_idx] = 0
    if mask.dim() > 0:
      true_dist.index_fill_(0,mask.squeeze(),0.0)
    self.true_dist = true_dist
    return self.criterion(x,nn.parameter(true_dist,requires_grad=False))

#Noam Schedaler

In [ ]:
class NoamScheduler(Lr_scheduler):
  def __init__(self,optimizer,model_size,fector,warmap,last_epoch =1):
    self.model_size = model_size
    self.fector = fector
    self.warmap = warmap
    super(NoamScheduler,self).__init__(optimizer,last_epoch)

  def get_lr(self):
    step = self.last_epoch + 1
    lr = self.fector *(self.model_size**(-0.5)*min(step**(-0.5),step*self.warmap**(-1.5))
    return [lr for _ in self.optimizer.param_groups]

#Create our model

In [ ]:
def make_model(src_vecab, tgt_vocab,N=6,d_model=512,d_ff=1024,h=8,droput=0.1,criterion = nn.BCELoss):
  c = copy.deepcopy
  attn = MultiHeadedAttention(h,d_model)
  ff = PositionwiseFeedForward(d_model,d_ff,dropout)
  position = position_encoding(d_model,dropout)
  model = EncoderDecoder(Encoder(EncoderLayer(d_model,c(attn),c(ff),dropout),N),
                         Decoder(DecoderLayer(d_model,c(attn),c(attn),c(ff),dropout),N),
                         nn.sequential(Embeddings(d_model,src_vocab),c(position)),
                         nn.sequential(Embeddings(d_model,tgt_vocab),c(position)),
                         Generator(d_model,tgt_vocab),criterion)
  for p in model.parameters():
    if p.dim()>1:
      nn.init.xavier_uniform_(p)
  return model